In [1]:
## ML pipeline
## DataSource -> Pre-Process -> InputData
## TrainingModule -> Model
## EvaluationData -> Pre-Process -> Model -> Result

## Detection pipeline
## [path] --> read -- [image, label] -->  crop -- [image, label]-->  hsv_hog --> [(hsv, hog), label] --> TrainingModule 

In [22]:
class FileSource:
    def __init__(self, src_dir, pattern, input_type='file'):
        self.src_dir = src_dir
        self.pattern = pattern
        self.files = glob.glob(os.path.join(self.src_dir, self.pattern), recursive=True)
        self.current = 0
        self.offset = 0
    
    def __iter__(self):
        return self
    
    def next(self):
        if self.current == len(self.files):
            raise StopIteration()
        else:
            file = self.files[self.current]
            self.current = self.current + 1
            return file
    
    def has_next(self):
        return self.current < len(self.files)
            

In [35]:
import os
import glob
import matplotlib.pyplot as plt

BASE_DIR = '/Users/raman/work/car_nd/self_driving_nano/p5_vehicle_detection'
TEST_INPUT_DIR = os.path.join(BASE_DIR, 'test_images')
INPUT_DIR = '/Users/raman/work/car_nd/self_driving_nano/p5_vehicle_detection/training_data/vehicles/GTI_Far'
#os.path.join(BASE_DIR, 'training_data')

def display_image(image):
    for image in pipeline.process():
        plt.imshow(image)
        plt.show()

plt.close()


In [30]:
from abc import ABCMeta,abstractmethod
import pickle

class Operation(metaclass=ABCMeta):
    
    def __init__(self):
        pass
        
    @abstractmethod
    def operate(self, input):
        pass

In [31]:
class SourceOperator:
    
    def __init__(self, data_source):
        self.data_source = data_source

    def next(self):
        while self.data_source.has_next():
            inp = self.data_source.next()
            yield inp
        
class ProcessingOperator:
    
    def __init__(self, src, operation):
        self.src = src.next()
        self.operation = operation
        
    def next(self):
        for inp in self.src:
            out =  self.operation.operate(inp)
            yield out
            
class CompositeOperator:
    
    def __init__(self, src, operations):
        self.src = src.next()
        self.operations = operations
        
    def next(self):
        for inp in self.src:
            outputs = []
            for op in self.operations:
                out =  op.operate(inp)
                outputs.append(out)
            yield np.concatenate(outputs) 
            
class OutputOperator:
    
    def __init__(self, src, writer):
        self.src = src.next()
        self.writer = writer
        
    def next(self):
        all_inputs = []
        for inp in self.src:
            all_inputs.append(inp)
        yield self.writer.operate(all_inputs)

In [32]:
class Pipeline:
    
    def __init__(self, output_operator):
        self.output_operator = output_operator
    
    def process(self):
        for output in  self.output_operator.next():
            yield output    
        

In [33]:
import cv2
import numpy as np

class ColorSpaceConverter(Operation):
    
    def __init__(self, color_space):
        self.color_space = color_space
        
    def operate(self, image):
        if self.color_space != 'BGR':
            if self.color_space == 'RGB':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            elif self.color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
            elif self.color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
            elif self.color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
            elif self.color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        else: feature_image = np.copy(image) 
        return feature_image

class Reader(Operation):
    
    def __init__(self):
        pass
        
    def operate(self, input_path):
        image = cv2.imread(input_path)
        return image
    
class Resizer(Operation):
    
    def __init__(self, size):
        self.size = size
   
    def operate(self, image):
        return cv2.resize(image, self.size)

class SpatialBinning(Operation):
    
    def __init__(self, size, color_space='RGB'):
        self.color_space = color_space
        self.size = size
    
    def operate(self, image):        
        # Convert image to new color space (if specified)
        if self.color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)             
        # Use cv2.resize().ravel() to create the feature vector
        features = cv2.resize(feature_image, self.size).ravel() 
        # Return the feature vector
        return features


class ChannelHistogram(Operation):

    def __init__(self, nbins=32, bins_range=(0, 256)):
        self.nbins = nbins
        self.bins_range = bins_range
    
    def operate(self, image): 
        return self.color_hist(image)

    # Define a function to compute color histogram features  
    def color_hist(self, img):
        # Compute the histogram of the color channels separately
        channel1_hist = np.histogram(img[:,:,0], bins=self.nbins, range=self.bins_range)
        channel2_hist = np.histogram(img[:,:,1], bins=self.nbins, range=self.bins_range)
        channel3_hist = np.histogram(img[:,:,2], bins=self.nbins, range=self.bins_range)
        # Concatenate the histograms into a single feature vector
        hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
        # Return the individual histograms, bin_centers and feature vector
        return hist_features


from skimage.feature import hog
class HOG:
    
    def __init__(self, orient, pix_per_cell, cell_per_block, hog_channel = 1):
        self.orient = orient
        self.pix_per_cell = pix_per_cell
        self.cell_per_block = cell_per_block
        self.hog_channel = hog_channel
    
    def operate(self, image):
        return self.hog(image)
    
    def hog(self, image):              
        if self.hog_channel == 'ALL':
            hog_features = []
            for channel in range(image.shape[2]):
                channel_feature = hog(image, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                      cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                      feature_vector=True)
                hog_features.append(channel_feature)
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = hog(image[:,:,self.hog_channel], orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                      cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                      feature_vector=True)
        return hog_features

from sklearn.preprocessing import StandardScaler
class StandardScaling:

    def __init__(self):
        pass
    
    def operate(self, feature_list):
        X = np.vstack(feature_list).astype(np.float64)
        # Fit a per-column scaler
        X_scaler = StandardScaler().fit(X)
        # Apply the scaler to X
        scaled_X = X_scaler.transform(X)
        return scaled_X

class PickleSerializer(OutputOperator):
    
    def __init__(self, output_path):
        self.output_path = output_path
    
    def operate(self, inp):
        with open(self.output_path, 'wb') as output:
            print("writing {} items to {}".format(len(inp), self.output_path))
            pickle.dump(inp, output)

class ImageDisplay(OutputOperator):
    
    def __init__(self):
        pass
    
    def operate(self, inp):
        plt.imshow(inp)
        plt.show()

    

In [36]:
from matplotlib import pyplot as plt

target_resize = (32, 32)
pix_per_cell = 8
cell_per_block = 2
orient = 9
spatial_bining_size = (32, 32)
output_file = 'feature_vectors_cars.p'

source        = FileSource(INPUT_DIR, "*.png")
path_provider = SourceOperator(source)
image_reader  = ProcessingOperator(path_provider, Reader())
resizer       = ProcessingOperator(image_reader, Resizer(target_resize))
hsv_converter = ProcessingOperator(resizer, ColorSpaceConverter('HSV'))
hsv_spa_hog   = CompositeOperator(hsv_converter, [ChannelHistogram(), SpatialBinning(spatial_bining_size), HOG(orient, cell_per_block, pix_per_cell)]) 
scaler        = ProcessingOperator(hsv_spa_hog, StandardScaling())
writer        = OutputOperator(scaler, PickleSerializer(output_file))
pipeline = Pipeline(writer)

for output in pipeline.process():
    pass
print("Done")

    

/Users/raman/anaconda/envs/carnd/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


writing 834 items to feature_vectors_cars.p
Done
